## 1. 환경 설정

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint

`(3) 벡터저장소 로드`  
- 저장해 둔 크로마 벡터저장소를 가져오기

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
#이미 Chroma db가 존재한다면 생성당시와 동일한 모델 선택
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="./chroma_db",
    )

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수: 11


## 2. LangChain LCEL

### 2.1 Prompt + LLM

In [4]:
# 다중 메시지 전송
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 모델 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.3, 
    max_tokens=100,
    )

messages = [
    ("system", "You are a helpful assistant."), # system 메시지 - 역할부여 
    ("user", "{query}"), # user - 외부에서 전달받는 사용자의 질문을 지정
]

# 메시지 리스트를 템플릿으로 변환
prompt = ChatPromptTemplate.from_messages(messages)

# 템플릿을 출력
print(prompt)

input_variables=['query'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]


In [5]:
# 템플릿 입력 변수를 출력
print(prompt.input_variables)

['query']


In [6]:
# input 값을 전달하여 프롬프트를 렌더링
# format method로 사전에 정의해둔 query 위치로 질문 바인딩
prompt_text = prompt.format(query="테슬라 창업자는 누구인가요?")

print(prompt_text)

System: You are a helpful assistant.
Human: 테슬라 창업자는 누구인가요?


In [7]:
# 모델에 prompt text를 직접 입력
response = llm.invoke(prompt_text)

# 모델의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)이라는 두 명의 엔지니어에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라의 초기 투자자로 참여한 후, 이후 CEO로서 회사를 이끌게 되었습니다.


In [8]:
# LCEL 체인을 구성 pipe 연산자로 prompt와 llm을 연결 -> chain을 구성
chain = prompt | llm

# 체인을 출력
print(chain)

first=ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]) last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001CAD8DE5690>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001CADA499810>, root_client=<openai.OpenAI object at 0x000001CABF87BE50>, root_async_client=<openai.AsyncOpenAI object at 0x000001CADA50D890>, model_name='gpt-4o-mini', temperature=0.3, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=100)


In [9]:
# 체인의 입력 스키마를 출력
from pprint import pprint
pprint(chain.input_schema.schema())

{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


In [10]:
# 체인을 실행 - 옵션 1
response = chain.invoke({"query":"테슬라 창업자는 누구인가요?"})

# 체인의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타르펋(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 성장에 큰 영향을 미쳤습니다. 따라서 엘론 머스크는 테슬라


In [11]:
# 체인을 실행 - 옵션 2 - 전달할 파라미터가 1개일 경우
response = chain.invoke("테슬라 창업자는 누구인가요?")

# 체인의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 이후 엘론 머스크가 2004년에 투자자로 참여하면서 CEO로 취임하고 회사의 방향성을 크게 이끌어왔습니다. 따라서 엘론 머스크는 테슬라의 가장


### 2.2 Prompt + LLM + Output Parser

`a) 문자열 파싱 - StrOutputParser`

In [12]:
response

AIMessage(content='테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 이후 엘론 머스크가 2004년에 투자자로 참여하면서 CEO로 취임하고 회사의 방향성을 크게 이끌어왔습니다. 따라서 엘론 머스크는 테슬라의 가장', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 27, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'length', 'logprobs': None}, id='run-11bef040-0ec0-4bf2-b8fc-056bbe7effb7-0', usage_metadata={'input_tokens': 27, 'output_tokens': 100, 'total_tokens': 127})

In [13]:
# StrOutputParser - 문자열 출력을 파싱
from langchain_core.output_parsers import StrOutputParser

# 출력 파서를 생성
output_parser = StrOutputParser()

# 출력 파서를 실행
output_parser.invoke(response)

'테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 이후 엘론 머스크가 2004년에 투자자로 참여하면서 CEO로 취임하고 회사의 방향성을 크게 이끌어왔습니다. 따라서 엘론 머스크는 테슬라의 가장'

In [14]:
str_chain = prompt | llm  | output_parser

query = "리비안의 설립년도는 언제인가요?"
str_response = str_chain.invoke(query)

print(str_response)



리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차를 개발하고 제조하는 기업으로, 특히 전기 픽업트럭과 SUV 모델에 주력하고 있습니다.


`b) JSON 출력 - JsonOutputParser`

In [15]:
from langchain_core.output_parsers import JsonOutputParser

# 출력 파서를 생성
json_parser = JsonOutputParser()

# 체인을 실행 (JSON 출력)
json_response = chain.invoke("테슬라 창업자는 누구인가요? JSON 형식으로 출력해주세요.") 
print(json_response)

# 출력 파서를 실행
json_parser_output = json_parser.invoke(json_response)
print(json_parser_output)

content='```json\n{\n  "창업자": "엘론 머스크",\n  "설명": "엘론 머스크는 테슬라의 공동 창립자이자 CEO로, 전기차 및 지속 가능한 에너지 솔루션을 개발하는 데 기여하였습니다."\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 34, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'stop', 'logprobs': None} id='run-392e9480-2003-4339-91d2-599fd8134bd7-0' usage_metadata={'input_tokens': 34, 'output_tokens': 62, 'total_tokens': 96}
{'창업자': '엘론 머스크', '설명': '엘론 머스크는 테슬라의 공동 창립자이자 CEO로, 전기차 및 지속 가능한 에너지 솔루션을 개발하는 데 기여하였습니다.'}


`c) Schema 지정 - PydanticOutputParser`

In [16]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator

# Pydantic 모델을 생성
class Person(BaseModel):
    """Information about a person."""   #필수 - 모델(이 담는 정보)에 대한 설명
#속성 정의 description 필수
    name: str = Field(..., description="The name of the person")
    title: str = Field(..., description="The title or position of the person.")

# 출력 파서를 생성
person_parser = PydanticOutputParser(pydantic_object=Person)
print("========================================")
print("PydanticOutputParser 프롬프트")
print("----------------------------------------")
print(person_parser.get_format_instructions())
print("========================================")


# Prompt 템플릿을 생성 - Pydantic 모델을 사용
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=person_parser.get_format_instructions())
#정의한 모델에 대한 정보를 프롬프트 내에 직접 전달하기 위해 format instructions 메소드 사용

print("Prompt 템플릿")
print("----------------------------------------")
print(prompt.format(query="테슬라 창업자는 누구인가요?"))
print("========================================")


PydanticOutputParser 프롬프트
----------------------------------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information about a person.", "properties": {"name": {"title": "Name", "description": "The name of the person", "type": "string"}, "title": {"title": "Title", "description": "The title or position of the person.", "type": "string"}}, "required": ["name", "title"]}
```
Prompt 템플릿
----------------------------------------
System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema

In [17]:
# 체인을 구성
person_chain = prompt | llm | person_parser

# 체인을 실행
response = person_chain.invoke("테슬라 창업자는 누구인가요?")

# 체인의 응답을 출력
response

Person(name='엘론 머스크', title='CEO 및 공동 창립자')

## 3. Chat Completion Methods

`(1) stream`  
- 입력에 대한 응답을 실시간 스트림을 생성하여 전달

In [18]:
import time 

for chunk in llm.stream("테슬라 창업자는 누구인가요?"):
    # 기본적으로 print 함수는 출력을 할 때마다 줄바꿈을 하지만, 줄바꿈 없이 출력하려면 end=""를 사용하면 됩니다.
    # flush=True 옵션을 사용하여 출력 버퍼를 즉시 비웁니다. 데이터를 지연 없이 즉시 출력하는 데 유용합니다.
    print(chunk.content, end="", flush=True)  
    time.sleep(0.1)  # 0.1초 대기 (100ms)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하르드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로 취임하며 회사의 성장에 중요한 역할을 하게 되었습니다. 이후 그는 테슬라의 비전과 방향

`(2) batch`  
- 입력 리스트에 대한 응답을 배치 단위로 생성

In [19]:
questions = [
    "테슬라의 창업자는 누구인가요?",
    "리비안의 창업자는 누구인가요?",
]

responses = llm.batch(questions)

for response in responses:
    response.pretty_print()
    print()

================================== Ai Message ==================================

테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프링어(Jeffrey B. Straubel) 등 여러 명이 있습니다. 그러나 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로서 회사의 방향성을 이끌어온 인물로 가장 잘 알려져 있습니다. 테슬라는

================================== Ai Message ==================================

리비안(Rivian)의 창업자는 RJ 스카링(RJ Scaringe)입니다. 그는 2009년에 리비안을 설립하였으며, 전기차 제조업체로서 주로 전기 픽업트럭과 SUV를 개발하고 있습니다. RJ 스카링은 MIT에서 박사 학위를 받았으며, 지속 가능한 운송 수단에 대한 비전을 가지고 회사를 이끌고 있습니다.



## 4. Runnable

`(1) RunnableParallel`

In [20]:
# 문서 검색기 생성
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 1}, 
)

query = "테슬라 창업자는 누구인가요?"
retrieved_docs = retriever.invoke(query) # 사용자 입력과 유사한 1개의 단일 문서를 list 형태로 return

retrieved_docs_text = "\n".join([doc.page_content for doc in retrieved_docs])

pprint(retrieved_docs_text)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.')


In [21]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

# RunnableParrellel 구성
runnable = RunnableParallel(
    {"context": itemgetter("context") , "question": itemgetter("question")}
)

# 객체를 실행
response = runnable.invoke({"context": retrieved_docs_text, "question": query})

# 응답을 출력
response

{'context': '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.',
 'question': '테슬라 창업자는 누구인가요?'}

`(2) RunnablePassthrough`

In [22]:
from langchain_core.runnables import RunnablePassthrough

runnable = RunnableParallel(    #Dictionary 형태 출력
    question=RunnablePassthrough(), #입력을 `그대로`` 전달
)

runnable.invoke({"query":"테슬라 창업자는 누구인가요?"})

{'question': {'query': '테슬라 창업자는 누구인가요?'}}

`(3) RunnableLambda`
- 정의: 파이썬의 커스텀 함수를 매핑하는데 사용

In [23]:
from langchain_core.runnables import RunnableLambda

def count_num_words(text):
    return len(text.split())    #공백 기준으로 분할

runnable = RunnableParallel(
    question=RunnablePassthrough(),
    word_count=RunnableLambda(count_num_words), #함수를 매핑
)

runnable.invoke("테슬라 창업자는 누구인가요?") 

{'question': '테슬라 창업자는 누구인가요?', 'word_count': 3}

## 5. 전체 RAG 파이프라인 구성

`(1) RAG 프롬프트 템플릿`  

In [24]:
# Prompt 템플릿을 생성
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

# 템플릿을 출력
prompt.pretty_print()

================================ Human Message =================================

Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]



`(2) Retriever Chain 연결`  

In [25]:
# 벡터 검색기
retriever = vectorstore.as_retriever(search_kwargs={'k': 2}) #2개의 문서 검색

# 문서 포맷터 함수
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs]) #검색된 문서들을 하나의 문자열로 결합

# 체인 구성
retriever_chain = retriever | format_docs

# 체인을 실행
response = retriever_chain.invoke("테슬라 창업자는 누구인가요?")

pprint(response)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010')


`(3) RAG Chain 연결`  

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=100)

# 체인 생성
rag_chain = (
    {"context": retriever_chain , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = rag_chain.invoke(query)

In [27]:
# 결과 출력
response

'마틴 에버하드와 마크 타페닝입니다.'

## 6. Gradio 챗봇

`(1) invoke 실행` 

In [28]:
import gradio as gr

def answer_invoke(message, history):
    response = rag_chain.invoke(message)
    return response

# Graiio 인터페이스 생성 
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

c:\Users\User\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-piYnTqFT-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\User\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-piYnTqFT-py3.11\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


In [29]:
# Graiio 종료
demo.close()

Closing server running on port: 7860


`(2) stream 실행` 

In [30]:
import gradio as gr

def answer_invoke(message, history):
    partial_message = ""
    for chunk in rag_chain.stream(message):         #토큰 단위로 출력
        if chunk is not None:
            partial_message = partial_message + chunk
            time.sleep(0.1)
            yield partial_message

# Graiio 인터페이스 생성 
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\User\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-piYnTqFT-py3.11\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


In [31]:
# Graiio 종료
demo.close()

Closing server running on port: 7860
